<p align="center">
  <img src="Graphics/Episode XI.png" />
</p>

## (0) Task & Motion Planning

### (0.1) Reminder: The Task Planning (TP) and Motion Planning (MP) Problems

**Informal TP Definition:** The problem of finding a sequence of actions for achieving a goal from an initial state assuming that actions have *deterministic* effects and that the environment is *fully observable*.

**Informal MP Definition:** Given a robot and a set of obstacles, plan a *collision-free* path from a start position to a goal position.

### (0.2) Learning Outcomes

In this tutorial, we will cover:
* The challenges of integrating task and motion planning for real robots
* A variety of approaches to the integrated TAMP problem 

<p align="center">
  <img src="Graphics/RobotDishwasher.gif" width=500/>
</p>

## (1) Overview of TAMP

Over the last 10 weeks or so, we've explored methods for teaching robots to make "high-level" decisions about complex tasks they'd like to accomplish as well as approaches for helping robots make "low-level" decisions about their motions in geometric space. It should be easy to just combine these two processes and build real robots that can plan and execute complex missions autonomously, right?

Think again!

### (1.1) Cognitive Robots in the Real World

One of the main focuses of the robotics community today is the development of robots that can be deployed in challenging, **unstructured**, object-rich and interaction-rich environments. In these applications, the robot will generally be tasked with a specific goal, such as cooking and delivering a meal to an elderly resident, but the actions necessary to achieve the goal will vary enormously depending on the state of the environment. For example, the robot might need to open cupboards and remove objects in order to retrieve a bowl that is necessary for preparing the meal. Directly specifying the full behavior policy for a robot operating in these unstructured environments is not practical because the required policy is too complex.

<p align="center">
  <img src="Graphics/RobotChef.gif" width=500/>
</p>

Up until now, we've explored two domains of planning that seek to allow robots to autonomously choose their own behavior policies for these kinds of environments: task planning and motion planning. Before even trying to combine these concepts, we see that each one presents their own complications when we consider real-world applications. We saw previously that task planning essentially boils down to being a search problem. Such search problems are often challenging and grow exponentially with the size of the problem, so it is no surprise that task planning is often symbolic: there are relatively few possible actions to choose from, with a relatively small set of finite parameters. Otherwise, search is prohibitively expensive even in the face of clever algorithms and heuristics.

The geometry-agnostic nature of task planning also proved a problem, and we tried to approach this issue as well through motion planning. We've only considered collision-free robot motion, though, which is important but does not enable the robot to alter the world. In order for the robot to, for example, move objects by picking them up and placing them, planning needs to consider a much larger space that encompasses the entire state of the world, which includes any objects the robot has grasped, the grasps it is using, and the poses of the other objects. Conceptually, it makes sense to try to directly extend motion planning methods to apply to entire world states, but this approach fails algorithmically. The entire world state, seen as a single kinematic system, is highly **under-actuated**, in the sense that from any configuration, most of the degrees of freedom cannot be changed at will. The robot can only change the position of an object by moving over and touching it.

<p align="center">
  <img src="Graphics/AtlasTryingItsBest.gif" width=500/>
</p>

This is where **Task and Motion Planning** (TAMP) comes in. What if our planner spends more effort deliberating about more concrete aspects of a plan before executing it? This presents a key tradeoff in more up-front computation, but a lower risk of failing at runtime. Research in TAMP seeks to combine AI approaches to task planning and robotics approaches to motion planning. A critical requirement for generality in approaches to TAMP actually lies between discrete “high-level” task planning and continuous “low-level” motion planning: an intermediate level of selecting the real-valued mode parameters, such as how to grasp and where to place an object, which govern legal continuous motions of the system. This class of problems is computationally difficult in theory and requires algorithmic sophistication in practice.

#### (1.1.1) Decoupled TAMP

The naive approach to this problem would be to just conduct task and motion planning separately: we first do the discrete task planning, and then we do the continuous motion planning to fill in the geometric gaps left by the task planner in its solution - this is called **Decoupled Task and Motion Planning** (DTAMP). The earliest algorithms for TAMP committed to this strict hierarchy of first finding an action sequence, and then finding continuous parameter values. For example, Shakey performed strips planning over high-level abstract actions, such as which room to move to, and then planned low-level motions that realized the high-level plan, with no mechanism for finding an alternative high-level plan if the lower-level motions were not possible. So, Shakey aggressively assumed that problems satisfy the **downward refinement** property. More formally, a two-level hierarchy satisfies the downward refinement property if every solution to the high level can be refined into a solution at the low level. When this property holds, problems can be completely disentangled into separate task planning and motion planning problems, so an algorithm that determines a plan skeleton based on values of discrete template arguments strictly before solving the associated constraint-satisfaction problem is complete.

<p align="center">
  <img src="Graphics/Shakey.jpg" width=500/>
</p>

In TAMP problems in practice, downward refinement rarely holds. As soon as geometric or kinematic considerations make some high-level plans infeasible (because, for example, three objects do not actually fit into the box we planned to put them in, or because the grasp needed to remove an object from a shelf will not work to place it on the stove and there is no surface available to use for regrasping), then we cannot inflexibly commit to any abstract plan without knowing its geometric and kinematic feasibility. In such cases, we'll need the help of **Integrated Task and Motion Planning** (ITAMP): simultaneous discrete & continuous planning! But first, we'll need to gather a bit more background knowledge...

<p align="center">
  <img src="Graphics/TAMP.png" width=600/>
</p>

### (1.2) Robot Manipulation

Before we dive further into TAMP, we first need to consider a whole new sub-field of robotics that we haven't explored before: **manipulation**. Manipulation extends the problem space of planning to include changing the state of other objects in the world (i.e. **movable/interactive obstacles**). To formalize manipulation problems, we need to model changes in the kinematics of the system, extend motion planning to handle constraints beyond collision avoidance, and integrate these components.

#### (1.2.1) Kinematic Graphs

One way to represent the geometric state of many environments is to encode the state variables collectively as a **kinematic graph**, which makes their dependencies explicit. In a kinematic graph, vertices represent bodies and the robot’s controllable joints, and edges represent attachments. Each edge has an associated relative transformation between the child body and parent body, which is a pose in *SE(3)* (this is a Lie group called the 3D Special Euclidean Group, and it represents the group of simultaneous rotations and translations for a vector). If each body is connected to at most one parent body and the graph is acyclic, this is a kinematic tree, for which the full state of the world can be derived from just the joint values of the robot *q* through forward kinematics. The attachments can be of several kinds. The most straightforward is a rigid attachment, which models an object resting stably on a surface or a robot grasping an object in a fixed grasp.

<p align="center">
  <img src="Graphics/KinGraph.png" width=600/>
</p>

The figure above represents a kinematic tree for an example kitchen environment that contains a robot manipulator with two joints ($j_{1}, j_{2}$), a fixed `Table` and `Stove`, and movable objects `Plate`, `Pizza`, and `Book`. Initially, `Pizza` rests on the `Plate`, which itself rests on the `Table`. When the robot picks up the `Plate`, it also transitively picks up `Pizza`. This change in the kinematic graph is referred to as a **kinematic switch**, which is a type of **mode switch**. After the switch, as the manipulator moves, the poses of both the `Plate` and `Pizza` change with respect to the world; we move through this mode using the same actuators as before, but the feasible configuration space has changed.

#### (1.2.2) Constrained Motion Planning

When the robot interacts with objects in the world, the effective configuration space is no longer the degrees of freedom of the robot: it corresponds to the state of the whole system; we denote this space $\mathcal{W}$. This state can be described by the discrete structure encoded in a kinematic graph, as well as continuous values of the transformations on the edges, which encode static relationships. However, these systems are generally *under-actuated*, meaning that they cannot be locally controlled in arbitrary directions, because we can only directly actuate the robot’s degrees of freedom. Despite this, we can indirectly manipulate these objects by controlling the robot. We begin by considering a simple “single-mode” problem in which the kinematic graph is fixed. The figure below illustrates a robot gripper pulling a drawer where the gripper pose is fixed relative to the drawer. Although normally the gripper can translate generally in $x,z$, the drawer only has a single degree of freedom, denoted by joint $j$. The combined configuration space of the gripper and the drawer is a three-dimensional space with coordinates $\langle x, z, j \rangle$, but they are constrained by the need to keep the gripper attached to the drawer.

<p align="center">
  <img src="Graphics/ConstrMP.png" width=600/>
</p>

In this and many other manipulation problems, the constraint function $F(w)$, which now applies to the whole world configuration $w \in \mathcal{W}$, includes both the collision-free constraint and a kinematic constraint, which causes $\mathcal{W}_{F}$, the subspace of $\mathcal{W}$ for which $F$ holds, to be lower dimensional than $\mathcal{W}$. As a result, sampling $\mathcal{W}$ randomly will have zero probability of producing a sample in $\mathcal{W}_{F}$, rendering standard sampling-based motion planning methods ineffective. This difficulty of sampling motivated the development of constrained motion planners, which explicitly take these constraints into account and plan within the low-dimensional space $\mathcal{W}_{F}$.

Dimensionality-reducing constraints are often expressed using a mode parameter $\sigma$, a fixed value that affects the constraint $F_{\sigma}(w)$. In general, $\sigma$ is real-valued. Here, we illustrate the effect of two different choices of this value, $\sigma_{1}$ and $\sigma_{2}$. Each stipulates a different rigid attachment pose between the gripper and the drawer handle. The graph in the figure above illustrates the combined configuration space and the feasible spaces $\mathcal{W}_{F_{\sigma_{1}}}$ and $\mathcal{W}_{F_{\sigma_{2}}}$, which are lines. The modes $\sigma_{1}$ and $\sigma_{2}$ allow motion of the gripper along different 1D lines in this 3D space, depending on the grasp of the drawer handle.

When the kinematic graph is a tree, the planning problem is relatively easy. The set of pairwise rigidity constraints specify all poses of objects relative either to the world frame (fixed objects) or to the robot (grasped objects). This collection of poses collectively constitutes a mode $\sigma$. We can sample full configurations for the system that exactly satisfy these constraints by simply sampling the robot’s degrees of freedom $q$, and performing forward kinematics to derive the full configuration $w$.

#### (1.2.3) Multi-Modal Motion Planning (MMMP)

Constrained motion planning provides a framework for reasoning about systems with many degrees of freedom, but few actuators. However, it assumes that the constraints themselves remain constant, and therefore is not expressive enough to model multi-step manipulation problems in which the robot must make and break contact, changing the kinematic graph, and therefore the active constraints on its motions. In order to model such problems, we must allow the mode to undergo discrete changes. The state of the system is $s = \langle w, \sigma \rangle$, where we can think of $w \in \mathcal{W}$ as the collective configurations of the robot and all other objects or mechanisms in the environment and $\sigma$ as the additional mode information, indicating for example which objects are currently attached to which others. For the most common cases of multi-modal motion planning (MMMP), we can refactor this representation, so that $s = \langle w, K \rangle$, where $q \in \mathcal{Q}$ is the robot configuration and $K$ is a kinematic graph which contains the mode
information and implies poses of all the bodies in the system, but we will make our general presentation in terms of $\langle w, \sigma \rangle$.

**Formal Definition:** A MMMP problem consists of a finite set $\{\Sigma_{1}, . . . , \Sigma_{m}\}$ of *mode families*, each of which has a real-valued parameter vector $\theta$. Associated with each mode $\sigma = \Sigma(\theta)$ is a constraint function $F_{\sigma}$ on full system configurations. At any given time, the system state $\langle w, \sigma \rangle$ is in a single mode $\sigma$, but whenever $w \in F_{\sigma '}$, the system may execute a **mode switch**, typically represented by a change to the kinematic tree, into mode $\sigma '$. The goal of an MMMP problem is typically a set of full system configurations $\mathcal{W}_{∗}$, and a solution has the form $[\sigma_{0}, \tau_{0}, \sigma_{1}, \tau_{1}, . . . , \sigma_{k}, \tau_{k}]$, where $s_{0} = \langle w_{0}, \sigma_{0} \rangle$ is the initial state of the system, $\tau_{i}$ is a trajectory in $w \in F_{\sigma_{i}}$, $\tau_{0}(0) = w_{0}$, $\tau_{i}(0) = \tau_{i−1}(1)$ for $i \in \{1, ..., k\}$, and $\tau_{k}(1)\in\mathcal{W}_{∗}$.

As an example, we model pick and place tasks in this framework. Modes in which the robot is not grasping any objects are **transit modes**, and modes in which the robot is
holding an object are **transfer modes**. For a robot with a single gripper, there is a transit mode family for free motion and a transfer mode family for each object that it can grasp. In the transit mode family, the mode parameter is comprised of the fixed world poses of every movable object. In the transfer mode family for a particular object, the mode parameter contains the grasp pose as well as the fixed world poses of every other movable object. Thus, although the system can only operate according to a single mode at a time, the mode parameter is high-dimensional because it contains constraints involving every movable object. For interactions with cyclic kinematic graphs, such as manipulating a drawer or opening a door, a constrained motion-planner is generally required in order to plan within the mode.

A key challenge in MMMP is identifying configurations that are in the intersection of the constraint sets for two modes and thus allow the system to switch
between them. This intersection is often lower dimensional than the feasible space $Q_{F_{\sigma}}$ of either mode. In a pick-and-place domain, in order to perform a kinematic switch between a transit and transfer mode, the robot’s gripper must be in contact with the involved object at a particular pose. This requirement imposes 6 constraints on the robot’s configuration, and as a result, the set of solutions is ($d − 6$)-dimensional. Fortunately, solutions can often be found using *inverse kinematics* (IK) either by projecting random samples into the constraint set using optimization or by analytically solving for the solutions to a reparameterized set that captures its underlying dimensionality. 

<p align="center">
  <img src="Graphics/MMMP_Ex.png" width=600/>
</p>

The figure above demonstrates a 1D robot (`R`) acting in the presence of a single movable object (`A`). The two plots visualize the 2D combined configuration space of the robot and the movable object. The left plot demonstrates the robot moving during a transit mode. The two 1D blue lines indicate the space for which the system can change, which depends on the current mode $\sigma_{1}$ or $\sigma_{2}$. These modes correspond to different placements of the movable object, which remains constant. The yellow region corresponds to infeasible states where the robot and the object are in collision. Because the object can be placed anywhere on the interval, there are infinitely many possible transit modes. The right plot demonstrates the robot and object moving during a transfer mode. The robot can attach itself either to the left or right side of the object. As a result, there are two possible transfer modes $\sigma_{1}^{A}$, $\sigma_{2}^{A}$, indicated by the 1D red lines. The relative pose between the robot and object remains constant during a transfer mode. The robot can switch between transit and transfer modes at a zero-dimensional (point) intersection between both lines ($q_{1}^{*}$, $q_{2}^{*}$). In order to switch to a new transit mode, the robot must first enter the appropriate transfer mode.

### (1.3) The TAMP Problem

Now that we have a basic understanding of robotic manipulation, we see that the configuration space of the world has important modal structure: depending on where the objects are placed and how they are grasped, the legal changes to the world are in a different mode or feasible submanifold of the full space. Furthermore, it is only possible to change modes by moving to an intersection of the feasible space of the current mode and a new one, which is in general an even lower-dimensional subspace. For these reasons, planning is best viewed as a *hybrid* discrete-continuous search problem, of selecting a finite sequence of discrete mode types (i.e. which objects to pick and place), continuous mode parameters (such as the poses and grasps of the movable objects), and continuous motion paths within each mode to a configuration that is in the intersection with the subsequent mode. With this understanding at hand, we can informally define the task and motion planning problem that we wish to tackle.

**Informal Definition:** TAMP is the problem of planning for a robot that operates in environments containing a large number of objects, taking actions to move itself through the world as well as to change the state of the objects.

<p align="center">
  <img src="Graphics/HybridSpec.png" width=600/>
</p>

#### (1.3.1) TAMP Problem Description

Informally, TAMP problems use compact representational strategies from task planning to describe and extend a class of MMMP problems. TAMP is an extension of MMMP in that there
may be additional state variables that are not geometric or kinematic, such as whether the lights are on or the pizza is cooked. We begin by articulating a generic MMMP, using an
extension of a task planning formulation, in the figure below. There are two extensions of the task planning formalism visible here. First, there are *continuous action parameters*. Second, in addition to preconditions and effects we have a new type of clause, called `con` for *constraint*. It is a set of constraints that all must hold true among the continuous parameters of the action in order for it to be a legal specification of a transition of the system.

<p align="center">
  <img src="Graphics/MMMP_TP.png" width=600/>
</p>

In the TP-style formalization of an MMMP problem, we see that there is a `moveWithin` action for each mode family $\Sigma_{i}$ and a `switchModes` action for each mode family pair $\Sigma_{i},\Sigma_{j}$. This formulation does not extend to the basic formulation of MMMP, but it provides a clear articulation of the overall system dynamics. In a domain with a large number of objects, there will be a large number of mode families, each of which requires specifying a constraint on a very high-dimensional world configuration space. What TAMP adds is the ability to “unpack” the entities in the problem description into sub-parts that are simpler to describe and that reveal substructure in the problem that enables algorithmic insights.

Consider an example which has five movable objects, `A` through `E`. We decompose the system configuration $w$ into state variables `atRob`, `holding`, `at[A]`, `at[B]`, `at[C]`, `at[D]`, and `at[E]`. The discrete state variable holding can take values ranging over `{None, A,B, C, D, E}` and specifies the current mode family $\Sigma$. The variable `atRob` is now a robot configuration, and `at[obj]` is the pose of object `obj` relative to either the world coordinate frame (when `holding` $\neq$ `obj`) or the robot hand coordinate frame (when `holding = obj`). The `moveF` (move while the gripper is free) and `moveH` (move while the gripper is holding) actions describe transit and transfer motion within modes. The `pick` action corresponds to a switch from a transit mode to a transfer mode, while the `place` action corresponds to a switch from a transfer mode to a transit mode.

<p align="center">
  <img src="Graphics/TAMP_Form.png" width=700/>
</p>

The sparsity of effect of planning action descriptions is a good match for articulating which state variables are changed (and, implicitly, which ones stay the same). We can see
that, in each action description, the `eff`: clause indicates just the variables that change. When the preconditions involve discrete constant values (such as `None`), they are being used to specify the mode family of the initial state of the transition. The advantage of being able to use templates is apparent: the `moveH` action has a template parameter `obj`, meaning that there is a mode family for each object being held.

Just as we have decomposed the configuration and the mode, we can decompose constraints, expressing them as conjunctions of constraints with smaller arity. For example, the `pick` action has the constraint `Kin[obj]`$(q, p, g)$. For any particular value of `obj`, representing an actual object in the domain, this represents a kinematic constraint, saying that if the robot is in configuration $q$ and holding object `obj` in grasp $g$, then `obj` will be at pose $p$. In `moveF` and `moveH`, the `Motion`$(q, \tau, q')$ constraint specifies the relationship between a trajectory $\tau$ and two robot configurations, asserting that $\tau(0) = q, \tau(1) = q'$, and $\tau$ is continuous. Notice that the trajectory $\tau$ appears neither in the preconditions nor effects of these actions; they are auxiliary parameters that describe motion *within* the modes.

The `Stable[obj]`$(p)$ constraint requires that $p$ be a pose representing a stable placement for object `obj` on a static object in the world. Similarly, the `OnStove[obj]`$(p)$ constraint requires that $p$ be a stable placement where `obj` is specifically on a stove. The `Grasp[obj]`$(g)$ constraint defines stable grasp poses (transforms between the hand frame and object frame) $g$ for object `obj`. This set may be finite if there only a few known grasps but could be uncountably infinite, in general. The collision-free constraint `CFreeA[obj]`$(p, g, \tau)$ asserts that if object `A` is at pose $p$, the robot is holding `obj` in grasp $g$, and it executes trajectory $\tau$, no collision will occur. The constraint `CFreeW[obj]`$(g, \tau)$ is defined similarly except that it involves the fixed objects in the world (indicated by the abbreviation `W`). Finally, although
not pictured, because the $p^{A}, . . . , p^{E}$ parameters in the `moveF` and `moveH` actions are each only mentioned in a single constraint and precondition, they can be compiled away using state constraints or inference rules (axioms), resulting in these actions templates being independent of the number of objects in the problem instance.

#### (1.3.2) Plan Skeletons & TAMP Solutions

In preparation for studying algorithms for solving TAMP problems, it is useful to examine the form of a solution, which is a finite sequence of action instances $\pi = [a_{1}, . . . , a_{k}]$, where each $a_{i}$ includes assigned values for all parameters that satisfy that action’s constraints. These actions induce a state sequence $[s_{0}, s_{1}, . . . , s_{k}]$, where each $s_{i}$ is a state expressed as an assignment of values to state variables, $s_{0}$ is the initial state of the problem, $s_{i−1}$ satisfies the preconditions of $a_{i}$, $s_{i}$ is the result of executing $a_{i}$ in $s_{i−1}$, and $s_{k}$ satisfies the goal conditions. Selecting the action templates and values for
the template variables specifies the form of a solution, which we call a **plan skeleton**. If the skeleton is fixed, the set of variables for which values must be selected is determined, and the problem that remains is one of selecting those values so that the constraints of the actions in the skeleton are satisfied.

Consider a TAMP problem with a single movable object `A`. Suppose the initial state is $s_{0} =$ `{atRob=`$\mathbf{q}_{0}$`, at[A]=`$\mathbf{p}_{0}$`, holding=None, cooked[A]=False}`, where the bold mathematical symbols $\mathbf{q}_{0}$, $\mathbf{p}_{0}$ are real-valued constants. The set of goal states can be defined using conditions and constraints, such as `cookedA=True`. One possible plan skeleton is:

<p align="center">
  <img src="Graphics/TAMP_sol.png" width=700/>
</p>

where $q_{1}, \tau_{1}, g_{2}, q_{2}, \tau_{3}, p_{4}$ are the free parameters. Plan skeletons can be visualized graphically by enumerating the sequence of $|\pi| + 1$ values of each state variable, as well as motion parameters $\tau_{1},\tau_{3}$ and associating the constraints of the $i$-th action with the appropriate $i − 1$ and $i$ state variables. Figure 8 illustrates this plan skeleton in the form of a **dynamic factor graph**. Round nodes represent state variables, and rectangular nodes represent constraints. Gray nodes have constant values, and colored nodes represent variables. Each vertical column corresponds to a state; the actions in the skeleton, which are responsible for the state changes are shown between the state columns, at the top. Multiple nodes of the same color represent a single variable that is constrained to maintain its current value across multiple steps of the plan. Each constraint is connected to the variables it constrains. Any assignment of values to the variables that satisfies all the constraints “fills out” the skeleton into a complete legal plan that is guaranteed to achieve the goal. However, it may be the case that no satisfying assignment exists.

<p align="center">
  <img src="Graphics/PlanSkel.png" width=800/>
</p>

#### (1.3.3) Hybrid Constraint Satisfaction

Finding an assignment of values to the parameters of a plan skeleton that satisfy the associated constraints is called a **hybrid constraint satisfaction problem** (H-CSP). Although many parameters are inherently continuous, some may have discrete domains. For example, there might be a finite set of stable resting surfaces for a particular object. The figure below compresses the plan skeleton from the figure above into a **constraint network**, a bipartite graph from parameters to constraints, by removing redundant constraints, constants, and parameters. Although TAMP is decidable via computational-geometry algorithms, just as in motion planning, most practical approaches use optimization or sampling to solve the underlying H-CSP's. Another dimension of variability in solution approaches is whether the method attempts to satisfy the entire constraint set at once or not: methods vary dramatically in their high-level control structure for handling the search over skeletons and parameter values, and make different demands on constraint satisfaction methods.

<p align="center">
  <img src="Graphics/ConstNet.png" width=700/>
</p>

##### (1.3.3.1) Joint Satisfaction

The most straightforward strategy for approaching an H-CSP is to reduce it to a constrained mathematical program and solve for values for all the free parameters at once. Although there is a vast literature on mathematical programming, solving programs corresponding to TAMP H-CSP's is often very difficult due to high dimensionality in continuous parameter space, the inclusion of discrete parameters, and the non-convexity of the constraints. There is no efficient, general solution method for these mathematical programs. There are, nonetheless, some approaches of practical value.

When all decision variables are real-valued, a common solution strategy is to minimize an objective function, which incorporates both the *hard* constraint violation and any soft action cost penalties, using local-descent methods, though these are guaranteed to reach only a *local* optimum of the objective function, which may not satisfy the constraints. The equation in the figure below displays a mathematical program corresponding to the constraint network we saw in the previous figure.

<p align="center">
  <img src="Graphics/jointsat.png" width=700/>
</p>

The trajectories $\tau_{1}, \tau_{3}$ are approximated as a sequence of robot configurations $\tau[0], \tau[1], ..., \tau[T]$ where $T$ is a hyperparameter. Each constraint is associated with a real-valued (and often once or twice differentiable) function, which is expressed either in an equality ($g(. . .)$) or inequality ($h(. . .)$) constraint for the mathematical program. Such optimization can also fluidly incorporate action costs, enabling it to identify a solution that is not only feasible but also low-cost. For example, the equation above minimizes the combined cost of moving through a `moveF` mode ($f_{moveF}(...)$) and a `moveH[A]` ($f_{moveH[A]}(...)$) mode, each of which are sums of a function defined on adjacent configurations that comprise trajectory parameter $\tau_{1}$ or $\tau_{3}$. More generally, **mixed-integer programming** (MIP) techniques are required. One prominent algorithm for solving MIPS is **branch-and-bound**, which performs a discrete search over assignments to the integer variables; then, conditioned on an assignment for each integer variable, the resulting mathematical program is real-valued and can be addressed by descent.

##### (1.3.3.2) Individual Satisfaction

An alternative approach to solving H-CSP's is to generate small groups of parameter values that satisfy a single constraint or a small set of constraints, and combine them. A **sampler** takes one or more constraints and generates a sequence of assignments of values to the free parameters, where each assignment that is generated is guaranteed to satisfy the constraints. A challenge when designing samplers is dealing with constraints whose set of satisfying values has lower dimension than combined domains of the free parameters. For example, the `Stable[obj]`$(p)$ constraint requires object `obj` to rest perpendicular to a 2D plane within a 3D pose space, so this constraint lies in *SE(2)* despite the set of object poses being in *SE(3)*. The rejection-sampling strategy of sampling at random from a bounded region of *SE(3)* will have zero probability of producing a value satisfying this constraint. However, samples can be produced by directly sampling `Stable` rather than *SE(3)*. Low-dimensional constraints remain problematic when attempting to produce values that also satisfy other constraints. For example, consider solving for values of $q, p, g$ that satisfy both `Stable[obj]`$(p)$ and `Kin[obj]`$(q, p, g)$. Here, the difficulty is finding a pose $p$ that satisfies `Stable` while also admitting values of $q, g$ that satisfy `Kin`. One solution is to explicitly design samplers that operate on larger collections of constraints; this approach generally reduces to the joint satisfaction approach.

Alternatively, one can design conditional samplers that take in input values for some of the parameters in the constraint(s) and produce satisfying output values for the rest of the parameters. Intuitively, these samplers consume values already known to satisfy some constraints and find completing values that are compatible for additional constraints. In the above example, a conditional sampler for `Kin[obj]` that takes in $p, g$ as inputs can consume a placement pose sampled by `Stable[obj]` and produce configurations $q$ through finding inverse kinematics (IK) solutions. In the event that no IK solution exists, the conditional sampler returns an empty sequence, effectively rejecting the input values. Boolean tests for a constraint can also be represented within this framework as degenerate “samplers” that perform a check on the input values but do not generate any output values. For example, the collision-free constraints `CFreeW`, `CFreeA`, and `CFreeW[A]` can be evaluated by querying a collision checker. In some applications, it may be beneficial to specify several conditional samplers for an individual constraint, which represent different partitions into input and output parameters. For example, an alternative sampler for `Kin[obj]` takes in $q, g$ and performs forward kinematics to produce a pose for `obj` that satisfies the constraint.

<p align="center">
  <img src="Graphics/samp_net.png" width=700/>
</p>

More generally, several conditional samplers can be composed to form a **sampling network**, a directed acyclic graph (DAG) defined on free parameters and conditional samplers. A
directed edge from a parameter to a sampler indicates that the parameter is an input to the sampler. A directed edge from a sampler to a parameter indicates that the parameter is an output of the sampler. Each parameter is required to be the output of exactly one sampler. The figure above gives an example sampling network for the factor graph we saw earlier.

### (1.4) General Approaches to TAMP

We now have the tools to search for action sequences and to solve H-CSP's. We can now begin to discuss strategies for combining them into Integrated TAMP algorithms. We would like to order the decision-making in a way that minimizes the overall runtime of the algorithm, for a problem distribution. There are several intuitive principles for organizing the search, which are sometimes in conflict with one another. One way to reduce search effort is to prune infeasible decision branches as quickly as possible (which is sometimes called **failing fast**). We would also prefer to postpone expensive computations until most of the rest of a potential solution is found. For example, in many manipulation applications, collision checking is expensive, due to the geometric complexity of 3D meshes and the need to check at a fine resolution to ensure safety, so we might wish to *lazily* postpone this operation. At the same time, we would like to balance the computational effort spent on each component, for example, by not spending too much time trying to satisfy the H-CSP associated with a single skeleton or even single constraint, in case it is unsatisfiable. Additionally, information gained in one branch of a high-level search, such as the solution or infeasibility of a subproblem, can often be re-used to make another branch of the search more efficient.

Let's examine the overall control flow of TAMP algorithms, which determines the relative ordering of action-sequencing and H-CSP subproblem solution. There are three predominant classes of strategies: **sequence before satisfy**, in which we find whole plan skeletons and then try to satisfy all of their constraints; **satisfy before sequence**, in which we find sets of satisfying assignments for individual constraints and attempt to assemble actions that use those values into complete plans; and **interleaved**, in which actions are added to the plan and additional constraints are satisfied incrementally. The following figure illustrates the first two classes of TAMP strategies as flowcharts:

<p align="center">
  <img src="Graphics/TAMP_Flow.png" width=800/>
</p>

## (2) Interleaved Approaches

There are many ways to **interleave** the searches for the action sequence and parameter values. In some cases, we would like to pre-sample state variable values, such as robot configurations and object poses, but defer the computation of motion parameter values, such as collision-free trajectories between two robot configurations. In this case, the domains of the state variables have already been discretized. Conditioned on an assignment of values to every non-motion parameter (state variable) for an action instance, each motion parameter is only affected by the constraints of that particular action, which means that the problem of finding satisfying values for its parameters is independent of finding parameters for other actions. Thus, the existence of a satisfying assignment to the motion parameters can be evaluated online during action sequencing in order to only compute values for action instances encountered during the search. The strategy was first applied to TAMP under the name of **semantic attachments**. Although this strategy limits the amount of interleaving that is possible, it is appealing in that the state space is fixed during sequencing; it only identifies that some transitions are infeasible.

Interleaved action and parameter search can also aid the search for plan skeletons when sequencing first. One source of search control is to observe that, in order for a plan skeleton to admit a satisfying assignment, all of its subsequences must also have satisfying assignments. Thus, a partial plan skeleton can be pruned from the search if its H-CSP is infeasible. Some approaches even solve relaxations of the induced H-CSP's that omit certain constraints such as motion constraints with many decision variables, which are often satisfiable and thus are uninformative.

A more general control structure is to perform a tree search, with layers alternating between selecting an action template and sampling parameter values for that action that satisfy the partial skeleton constraints. A substantial difficulty in this approach is that tree nodes may have infinitely many successor nodes due to the possibly infinitely many satisfying parameter values and action instances that could be performed. Thus, it is important that the search be *persistent*, in the sense that it will revisit previous search nodes indefinitely in order to generate additional samples for continuous parameters.

### (2.1) aSyMov [[2005]](https://hal.laas.fr/hal-01972666/file/2003-isrr-gravot.pdf)

**aSyMov** (a Symbolic Move3d) is a forward-search planner in the state space, and it relies on an approach based on PRM's. In TAMP, we must consider that picking and placing objects are discrete events which change the robot (we call this change "composition"). Different **composed** versions of the robot have different C-spaces, and so the main idea of aSyMov is that for each of these versions we will maintain separate (projected) PRM's which can be linked based on actions such as `pick` and `place`. For example, once the robot picks up an object and becomes composed, we will use the PRM for this composed robot to conduct its motion planning.

We will represent the location of robots and objects using a symbolic type called “position”, which corresponds to a set of configurations. For instance, the configurations where a robot `R` can grasp an object `O` will be denoted by `P_TI_TA_O_R`. Such a symbolic position corresponds to a transition between a transit motion (`TI`) and a transfer motion (`TA`). As a consequence of the grasp action, a new robot `R-O` will be created. It may place (transition between `TA` and `TI`) the object at a position denoted by `P_TA_TI_R-O`.

Where does task planning come in? When using aSyMov, we will specify the actions and goal state using a PDDL-like language, and we will utilize symbolic task plans as heuristics in the PRM expansion step.

<p align="center">
  <img src="Graphics/aSyMov.png" width=600/>
</p>

To define states and actions for aSyMov, we will first say that each state $s$ has two components:
- `sym`$(s)$: symbolic state (e.g. PDDL state)
- `geom`$(s)$: continuous state in the combined configuration space

<p align="center">
  <img src="Graphics/asymov_graph.png" width=300/>
</p>

A state is thus defined as a tuple $s=\langle w, w^{\#}\rangle $, where $w=$`sym`$(s)$ and $w^{\#}=$`geom`$(s)$. The position predicates in `sym`$(s)$ denote subsets of the configuration space $\mathcal{C}$, for example `P_FORKLIFT_CANGRASP_CBOX`. The operator $R$ maps each atomic position predicate to a subset of $\mathcal{C}$, for example $R($`on_table(`$b_{1}$`)`$) = \{c \in \mathcal{C}: b_{1}$ is on table in $\mathcal{C}\}$. Every symbolic state $w$ defines a subset of the C-space (i.e. $R(w) := \cap\{R(p) : p$ is a true, grounded location predicate in $w\}$.

An example of an action for aSyMov might look as follows:

<p align="center">
  <img src="Graphics/asymov_action.png" width=500/>
</p>

Here, `P_` again denotes position predicates, `P_FORKLIFT_CANGRASP_CBOX` denotes positions from where the `CBOX` object can be grasped, and `P_CBOX` indicates that `CBOX` is at some location, not grasped by robot. 

Now that we have the notions of states and actions defined for aSyMov, we can get more insight into the search process:

- Associate each action with a PRM (joint C-space, initial & goal configurations)
- Maintain a search tree of symbolic states with a set of sampled configurations for each
-  Two forms of expansion of a state $w$ with an action $a$
    - Apply $a$ (search for $a^{\#}$ of any member of $w$)
      - Generate child logical state $w' = a(w)$
      - If $w'$ is valid $(R(w') \neq \emptyset)$, add $w'$ as child of $w$
    - Expand the PRM for $a$ on $w$, recheck validity of $w'$
- Use a heuristic to figure out state, action, and nature of expansion 

<p align="center">
  <img src="Graphics/asy1.png" width=200/>
  <img src="Graphics/asy2.png" width=195/>
  <img src="Graphics/asy3.png" width=202/>
</p>

What heuristic do we use? For selecting the action to be applied on state $w$, we use: $h(a) =$ `plan_cost` + `failed_refinement_count`
- `plan_cost` = length of symbolic plan $\pi$ for reaching goal from $w' = a(w)$ (the only use of a symbolic planner and symbolic plan)
- `failed_refinement_count`: Using a search algorithm to find geometric states corresponding to each state in $\pi$, `failed_refinement_count` is the metric over this algorithm 

For selecting the state to expand, we use the `depth from the root` + `#failures` + `#cost of actions with no children`.

We should note that the aSyMov algorithm is *probabilistically complete* (thanks to the PRM's)!

### (2.2) Other Algorithms

**Planning with Semantic Attachments (PSA)** [[2009]](https://www.aaai.org/ocs/index.php/ICAPS/ICAPS09/paper/viewFile/754/1101): Integration of **semantic attachments** (the attachment of an interpretation to a predicate symbol using an external procedure) into a forward-chaining planner (originally applied to Fast Forward [FF] and Temporal Fast Downward [TFD]), as an extension of PDDL. This allows the evaluation of grounded predicates as well as the change of fluents by externally specified functions (i.e. condition-checker modules, effect-applicator modules, etc.).

**Hierarchical Planning in the Now (HPN)** [[2011]](https://people.csail.mit.edu/lpk/papers/hpnICRA11Final.pdf): A regression-based symbolic planner using generators, which perform fast approximate motion planning, to select geometric parameters, such as configurations and paths, for the actions. Reasoning backward using **regression** (*not* the same regression as in machine learning!) allows the goal to significantly bias the actions that are considered (i.e. we conduct backward chaining to identify relevant actions). It is aggressively **hierarchical**; it makes choices and commits to them in a top-down fashion in an attempt to limit the length of plans that need to be constructed, and thereby exponentially decrease the amount of search required. It operates on detailed, continuous geometric representations and does not require a-priori discretization of the state or action spaces.

## (3) Satisfy Before Sequence Approaches

An alternative strategy is motivated by the fact that task planning in finite domains can often be very efficient in even very large problem instances, and therefore seeks to reduce the hybrid problem of tamp to one or more discretized planning problems by generating values of continuous quantities, such as poses and configurations, and computing in advance which constraints they satisfy. For example, one might sample, for an environment with some fixed support surfaces, a set of values $p_{i}$ such that `Stable[A]`$(p_{i})$ holds. Approaches that perform satisfaction first almost always use individual satisfaction, which is typically implemented using sampling, because they aim to generate values that are useful for a variety of plan skeletons. A single round of sampling will in general not suffice. When the discrete planning problem given a particular set of values is infeasible, it is necessary to generate more samples and try again, as illustrated in the figure below (which we saw previously).

<p align="center">
  <img src="Graphics/SatThenSeq.png" width=700/>
</p>

Satisfying before sequencing is advantageous when the computational effort of repeatedly sequencing and failing to satisfy the associated H-CSP outweighs the computational
effort of eagerly generating values that satisfy constraints upfront. This often is the case when one or more of the following are true:   
- Sampling is efficient and does not result in a combinatorial explosion of sampled values
- Each discrete action sequencing search has non-negligible overhead
- Sampled values are unlikely to satisfy critical constraints

### (3.1) Probabilistic Tree of Roadmaps (PTR) [[2010]](http://motion.cs.illinois.edu/papers/aaai10-taskmotion.pdf)

Given a planning problem $\mathcal{P}$, the **Probabilistic Tree of Roadmaps** (PTR) algorithm performs a probabilistic forward search to
build a tree $T$ of feasible states, but uses (1) a density-weighting strategy to achieve an even coverage of state space, and (2) incrementally increases the time spent on individual motion planning queries so that hard queries will eventually be solved. PTR builds a tree $T$ with the following properties:
- An edge $x \rightarrow x'$ is constructed by selecting an action $A(u)$ that is applicable to $x$, and determining the resulting state $x'$. We attach the action $A(u)$ to the edge, as well as an (initially) incomplete motion planning query. The continuous action parameters $u$ are sampled for each state $x$.
- The status of an edge is marked as Complete or Active. Complete edges have finished a motion planning query, while motion planning queries are currently being processed at Active edges.
- If $T$ contains a path of Complete edges leading to a goal state, then the concatenation of trajectories along this path is a solution to the planning problem.

The figure below illustrates the growth of the search tree in (a) an abstract state space with regions of easy, hard, and infeasible
motion planning queries (white, grey, and black nodes). Active edges are dotted, Complete edges are solid. (b,c) After a few iterations, the easy portion of space is partially explored. (d) Continued exploration. (e) As the time cutoff for motion queries increases, more hard queries begin to be answered. (f) Reaching the goal.

<p align="center">
  <img src="Graphics/PTR.png" width=500/>
</p>

Motion planning queries are usually solved by sampling-based motion planning methods such as PRM and RRT (this allows the PTR algorithm to maintain *probabilistic completeness*, under some weak assumptions). We can also examine the pseudocode of the PTR algorithm below; notice that we are not doing any real task planning (the `Plan-More`$(e,t)$ subroutine performs more computation on the motion planning query associated with the edge $e$, and halts when the total time spent on this query since edge creation exceeds $t$)!

<p align="center">
  <img src="Graphics/PTR_Pseudo.png" width=400/>
</p>

While we don't explicit do any task planning, we will use a STRIPS-like language to describe actions and state variables (i.e. fluents) for the TAMP problems tackled by the PTR algorithm. We define an action $A(u)$ as a parameter-dependent tuple $\langle P, Q, E, C \rangle$ containing:
- Preconditions $P$: a list of boolean conditions that must be true of the current state $x$ before an action can be executed. If they are true, we say $A(u)$ is applicable to $x$.
- Queries $Q$ (optional): probabilistic roadmap motion planning queries that must be solved before the action is executed. Queries are written in the form `trajectory = Query(space,start,end)`. The `space` variable defines the continuous subspace of $\mathcal{X}$ in which the query is allowed to move, as well as the operational constraints for the query (e.g., obstacle avoidance, differential constraints).
- Effects $E$: changes to the state of the world once the action is executed, written in the form `Variable=value`. We assume that the effects of an action are invariant to the trajectory produced by a plan. Define the function $Appl(A(u), x)$ that applies the effects of $A(u)$ to the state $x$, and produces the resulting state.
- Commands $C$: a sequence of commands that should be sent to the robot upon executing this action.

As an example, let us consider the specification of a realistic implementation of the traditional “blocks world” problem that is often used in discrete planning benchmarks. The planner should produce kinematic manipulation plans for a manipulator arm that can pick and place any of $n$ rigid objects in a static environment. For simplicity, we will not allow objects to be stacked on top of one another. In this example, $\mathcal{X}$ contains the following state variables:
- `Robot`: continuous robot configuration in $\mathcal{C}$
- $\text{Object}_{i}$: ungrasped object pose in *SE(3)*, $i = 1, . . . , n$
- `GraspedObject`: the index of the grasped object
- `Grasp`: a description of the current grasp, i.e. the gripper width and its transformation relative to the object

<p align="center">
  <img src="Graphics/PTR_Actions.png" width=500/>
</p>

Motion planning queries are needed to generate transit (no object grasped) and transfer (object grasped) paths for the robot arm. Transit paths occur in the subspace of the robot configuration only, but collision constraints are dependent on the pose of the objects in the current state. Thus we write the collision-free transit subspace as $\mathcal{F}_{transit,x}$. In transfer motions the currently grasped object is rigidly affixed to the robot’s gripper, and thus the planner must perform collision detection between the grasped object against the environment and all other objects. We write this transit subspace as $\mathcal{F}_{transfer,x}$. The problem is defined using two actions, listed in the figure above, which make use of the following subroutines:
- `Object-Stable?`$(i, p)$ returns true if object $i$ is stably supported by the environment at pose $p$
- `Grasp-Stable?`$(i, g)$ returns true if object $i$ can be stably grasped using grasp $g$ (e.g. with a force closure grasp)
- `Pose`$(i, q, g)$ produces the pose of object $i$ if grasped by grasp $g$ at configuration $q$

### (3.2) Other Algorithms

**FFRob** [[2014]](https://dspace.mit.edu/bitstream/handle/1721.1/112348/Lozano-Perez_FFRob.pdf?sequence=1&isAllowed=y): This isn't exactly an algorithm per se, but rather an extension of the heuristic used in the FastForward (FF) planning system to a new heuristic (called **FFRob**) which takes geometric information into account. The FFRob heuristic integrates reachability in the robot configuration space with reachability in the symbolic state space, and both the search and the computation of FFRob exploit a roadmap data structure that allows multiple motion-planning queries on the closely related problems that arise during the search to be solved efficiently.

**Compilation of TAMP into Classical AI Planning** [[2017]](https://arxiv.org/pdf/1706.06927.pdf): This algorithm relies on pre-discretization for compiling TAMP problems into classical AI planning problems, i.e. planning problems over finite and discrete state spaces with a known initial state, deterministic actions, and goal states to be reached.
The compilation is sound, meaning that classical plans are valid robot plans, and probabilistically complete, meaning that valid robot plans are classical plans when a sufficient number of configurations is sampled. In this approach, motion planners and collision checkers are used for the compilation, but not at planning time. The key elements that make the approach effective are (1) expressive classical AI planning languages (*not* PDDL) for representing the compiled problems in compact form, that unlike PDDL make use of functions and state constraints, and (2) general width-based search algorithms capable of finding plans over huge combinatorial spaces using weak heuristics only.
- Think of how we solved the Travelling Jedi Problem using classical planning, where the geometric constraints were already accounted for as logical predicates / numeric fluents

## (4) Sequence Before Satisfy Approaches

As we mentioned earlier, the earliest algorithms for TAMP committed to a strict hierarchy of first finding an action sequence, and then finding continuous parameter values (for example, Shakey); such algorithms assumed that their problems satisfied the *downward refinement* property. We also noted that for TAMP problems in practice, downward refinement rarely holds. Even when downward refinement does not hold, though, a top-down problem decomposition can still be very effective, as long as there is a mechanism for “backtracking” and trying alternative high-level plans when the lower-level solver fails.

<p align="center">
  <img src="Graphics/SeqThenSat.png" width=700/>
</p>

The figure above (which we saw previously) illustrates this approach, in which there is an outer loop representing a search over legal plan skeletons; for each plan skeleton, we attempt to solve the associated H-CSP and if we succeed, we return the complete solution; otherwise, we return to the outer loop and try another skeleton. In many everyday TAMP applications, action sequencing is relatively inexpensive, making it advantageous to find a plausible action sequence before satisfying constraints. Furthermore, solving H-CSP's can be computationally expensive, so by only attempting to solve H-CSP's that correspond to viable plan skeletons, we can potentially save substantial computation time.

### (4.1) PDDLStream [[2020]](https://arxiv.org/pdf/1802.08705.pdf)

The key idea behind the **PDDLStream** algorithm is that it extends PDDL with the notion of **streams**. Streams are generic, user-defined programmatic (i.e. Python) functions that sample continuous parameters such that a valid sample certifies that stream and provides any necessary predicates that (usually) act as preconditions for actions. PDDLStream also has an adaptive technique that balances *exploration* (searching for discrete task plans) vs. *exploitation* (sampling to fill in continuous parameters). Please watch [this video](https://www.youtube.com/watch?v=_w8AbakN4uo) to get a better idea of how PDDLStream works, and read [this article](https://roboticseabass.com/2022/07/30/integrated-task-and-motion-planning-in-robotics/) to see an implementation of PDDLStream using a Python library called `pyrobosim`.

<p align="center">
  <img src="Graphics/pddlstream.png" width=505/>
  <img src="Graphics/stream.png" width=600/>
</p>

### (4.2) Other Algorithms

**Hierarchical Task Networks for Mobile Manipulation (HTN-MM)** [[2010]](https://people.eecs.berkeley.edu/~russell/papers/icaps10-hla.pdf): Application of [Hierarchical Task Networks](https://pages.mtu.edu/~nilufer/classes/cs5811/2012-fall/lecture-slides/cs5811-ch11b-htn.pdf) (HTN) to planning domains with continuous operators. Key idea: provide high-level actions, and their refinements as sequences of low-level operations on continuous variables. The novel features of the system are (1) it finds high-quality kinematic solutions to task-level problems, and (2) it takes advantage of subtask-specific irrelevance information, reusing optimal solutions to state-abstracted subproblems across the search space.
 
**Symbolic Interface for TAMP** [[2014]](https://people.eecs.berkeley.edu/~russell/papers/icra14-planrob.pdf): Maintain a graph of plans, and when expanding each plan-node we can either continue to search for feasible instantiations in existing node (using a motion planner) or replan to fix errors for an infeasible instantiation (using a task planner). The novelty here lies in the approach for communicating relevant geometric information to the task planner in terms of logical predicates, without discretization, using **symbolic references** to continuous values, such as “grasping pose for $b_{1}$” and “trajectory for reaching grasping pose for $b_{1}$”. This algorithm can use off-the-shelf task planners and motion planners and makes no assumptions about their implementation.

**Logic-Geometric Programming (LGP)** [[2015]](https://www.ijcai.org/Proceedings/15/Papers/274.pdf): Integrates a logical layer and a geometric layer in an **optimization** formulation. The logical layer describes feasible high-level actions at an abstract symbolic level, while the geometric layer uses continuous optimization methods to reason about motion trajectories with geometric constraints. See [this paper](https://karpase.net.technion.ac.il/files/2022/03/ICAPS_2022_Ortiz_et_al.pdf) to learn about Erez's recent work on LGP.

<p align="center">
  <img src="Graphics/AtlasCleaning.gif" width=500/>
</p>

## (5) Conclusion

In this tutorial, we:
* Introduced task and motion planning (TAMP) problems
* Discussed the fundamental challenges that necessitate ITAMP algorithms
* Covered some prominent TAMP methods, such as aSyMov, PTR, and PDDLStream

This was our last in-person tutorial together! Next week, there will be an asynchronous tutorial on reinforcement learning and probabilistic planning (serving as enrichment to the course material). Thank you guys for a great semester!

#### ***Credit:** This tutorial was written by Yotam Granov, Winter 2022.*

### **References**

[1] S. Cambon, R. Alami, and F. Gravot. ["A hybrid approach to intricate motion, manipulation and task planning"](https://hal.laas.fr/hal-01976081v1/file/2009-ijrr-cambon-report.pdf), 2009. *International Journal of Robotics Research*.

[2] S. Castro. ["Integrated Task and Motion Planning in Robotics"](https://roboticseabass.com/2022/07/30/integrated-task-and-motion-planning-in-robotics/), 2022. *Robotic Sea Bass*.

[3] C. Dornhege, P. Eyerich, T. Keller, S. Trug, M. Brenner, and B. Nebel. ["Semantic Attachments for Domain-Independent Planning Systems"](https://www.aaai.org/ocs/index.php/ICAPS/ICAPS09/paper/viewFile/754/1101), 2009. *International Conference on Automated Planning and Scheduling (ICAPS)*.

[4] J. Ferrer-Mestres, G. Frances, and H. Geffner. ["Combined Task and Motion Planning as Classical AI Planning"](https://arxiv.org/pdf/1706.06927.pdf), 2017.

[5] C. Garrett. ["Task and Motion Planning Tutorial"](http://web.mit.edu/caelan/www/presentations/CRSS19.pdf), 2019. *3rd Cognitive Robotics Summer School, University of Southern California*.

[6] C. Garrett, R. Chitnis, R. Holladay, B. Kim, T. Silver, L. Kaelbling, and T. Lozano-Perez. ["Integrated Task and Motion Planning"](https://arxiv.org/pdf/2010.01083.pdf), 2021. *Annual Review of Control, Robotics, and Autonomous Systems*.

[7] C. Garrett, T. Lozano-Perez, and L. Kaelbling. ["FFRob: An efficient heuristic for task and motion planning"](https://dspace.mit.edu/bitstream/handle/1721.1/112348/Lozano-Perez_FFRob.pdf?sequence=1&isAllowed=y), 2014. *Workshop on the Algorithmic Foundations of Robotics*.

[8] C. Garrett, T. Lozano-Perez, and L. Kaelbling. ["PDDLStream: Integrating Symbolic Planners and Blackbox Samplers"](https://arxiv.org/pdf/1802.08705.pdf), 2020. *International Conference on Automated Planning and Scheduling (ICAPS)*.

[9] F. Gravot, S. Cambon, and R. Alami. ["aSyMov: a planner that deals with intricate symbolic and geometric problems"](https://hal.laas.fr/hal-01972666/file/2003-isrr-gravot.pdf), 2005. *International Symposium on Robotics Research*.

[10] K. Hauser. ["Task planning with continuous actions and nondeterministic motion planning queries"](http://motion.cs.illinois.edu/papers/aaai10-taskmotion.pdf), 2010. *Proceedings of AAAI Workshop on Bridging the Gap between Task and Motion Planning*.

[11] L. Kaelbling and T. Lozano-Perez. ["Hierarchical task and motion planning in the now"](https://people.csail.mit.edu/lpk/papers/hpnICRA11Final.pdf), 2011. *IEEE International Conference on Robotics and Automation (ICRA)*.

[12] J. Ortiz-Haro, E. Karpas, M. Toussaint, and M. Katz. ["Conflict-Directed Diverse Planning for Logic-Geometric Programming"](https://karpase.net.technion.ac.il/files/2022/03/ICAPS_2022_Ortiz_et_al.pdf), 2022. *International Conference on Automated Planning and Scheduling (ICAPS)*.

[13] S. Srivastava, E. Fang, L. Riano, R. Chitnis, S. Russell, and P. Abbeel. ["Combined Task and Motion Planning Through an Extensible Planner-Independent Interface Layer"](https://people.eecs.berkeley.edu/~russell/papers/icra14-planrob.pdf), 2014. *IEEE International Conference on Robotics and Automation (ICRA)*.

[14] S. Srivastava and S. Patil. ["Task and Motion Planning for Robots in the Real World"](https://www.youtube.com/watch?v=wRZ2yqRrPiY), 2014. *ICAPS 2014 Tutorial*.

[15] M. Toussaint. ["Logic-geometric programming: an optimization-based approach to combined task and motion planning"](https://www.ijcai.org/Proceedings/15/Papers/274.pdf), 2015. *International Joint Conference on Artificial Intelligence*.

[16] J. Wolfe, B. Marthi, and S. Russell. ["Combined Task and Motion Planning for Mobile Manipulation"](https://people.eecs.berkeley.edu/~russell/papers/icaps10-hla.pdf), 2010. *International Conference on Automated Planning and Scheduling (ICAPS)*.